In [1]:
import pandas as pd 
import numpy as np

In [2]:
df = pd.read_csv('Walmart.csv')
#print(df.head())

In [3]:
target_col = 'Weekly_Sales'
num_cols = [ 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
cat_cols = ['Store', 'Holiday_Flag']
feature_cols = cat_cols + num_cols
# может перевести номер филиала в категориальный признак ?

# что делать с DATE numerical? categorical?

# Divide on train and test

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import mean_squared_error, r2_score



In [5]:
one_hot_encoded = pd.get_dummies(df, columns=cat_cols)
df = one_hot_encoded
#df = pd.concat([df, one_hot_encoded], axis=1)
#df.drop(columns=cat_cols, axis=1, inplace= True)
#print(df)

In [6]:
#dataset = Dataset(X_tensor, y_tensor)

y = df[target_col]
X = df.drop(columns=target_col, axis=1)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
mean = X_train[num_cols].mean()
std = X_train[num_cols].std()

# Standardize numerical features
X_train[num_cols] = (X_train[num_cols] - mean) / std
X_test[num_cols] = (X_test[num_cols] - mean) / std

In [9]:
#print(X_test)

In [10]:
X_train.drop(columns='Date', axis=1, inplace= True)
X_train = torch.tensor(X_train.values, dtype=torch.float32) 
y_train = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1) 

X_test.drop(columns='Date', axis=1, inplace= True)
X_test =  torch.tensor(X_test.values, dtype=torch.float32) 
y_test = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)



In [11]:
X_test = X_test.to('cuda')
X_train = X_train.to('cuda')
y_test = y_test.to('cuda')
y_train = y_train.to('cuda')

In [15]:
input_size = 51
output_size = 1
learning_rate = 0.1
num_epochs = 3
batch_size = 16


# Создание простой модели с одним линейным слоем
model = nn.Sequential(
    nn.Linear(input_size, output_size)
)

# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

num_samples = X_train.shape[0]

model.to('cuda')
train_loss = torch.zeros_like(y_train, device='cuda')
train_pred = torch.zeros_like(y_train, device='cuda')
batch_indices = torch.empty((batch_size), device='cuda')
batch_X = torch.empty((batch_size), device='cuda')
batch_y = torch.empty((batch_size), device='cuda')

# Обучение модели
for epoch in range(num_epochs):
    for batch_start in range(0, num_samples, batch_size):
        batch_X = X_train[batch_start:batch_start+batch_size]
        batch_y = y_train[batch_start:batch_start+batch_size]

        train_pred = model(batch_X)
        train_loss = criterion(train_pred, batch_y)

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()    


with torch.no_grad():
    test_output = model(X_test)
    print("Test output:", r2_score(test_output.cpu().detach().numpy(), y_test.cpu().detach().numpy()))

Test output: 0.9043387137573216


In [18]:
input_size = 51
output_size = 1
learning_rate = 0.1
num_epochs = 3
batch_size = 16

# Создание простой модели с одним линейным слоем
model = nn.Sequential(
    nn.Linear(input_size, output_size),
    nn.ReLU()
)
model.to('cuda')

# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
num_samples = X_train.shape[0]

# Обучение модели
for epoch in range(num_epochs):
    for batch_start in range(0, num_samples, batch_size):
        batch_X = X_train[batch_start:batch_start+batch_size]
        batch_y = y_train[batch_start:batch_start+batch_size]

        train_pred = model(batch_X)
        train_loss = criterion(train_pred, batch_y)

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()    


with torch.no_grad():
    test_output = model(X_test)
    print("Test output:", r2_score(test_output.cpu().detach().numpy(), y_test.cpu().detach().numpy()))

Test output: 0.9043404423618708


In [19]:
print(y_test)

tensor([[1653759.3750],
        [ 597644.0000],
        [ 281699.6875],
        ...,
        [1162042.2500],
        [1087507.5000],
        [ 398445.1562]], device='cuda:0')


In [119]:
def trim_to_hight(layer):
    if isinstance(layer, nn.Linear):
        max_val = 100_000
        min_val = -max_val
        
        layer.weight.data = torch.where(layer.weight.data > max_val, torch.tensor(max_val), layer.weight.data)
        layer.weight.data = torch.where(layer.weight.data < min_val, torch.tensor(min_val), layer.weight.data)
        layer.bias.data = torch.where(layer.bias.data > max_val, torch.tensor(max_val), layer.bias.data)
        layer.bias.data = torch.where(layer.bias.data < min_val, torch.tensor(min_val), layer.bias.data)
 


input_size = 51
hiden_size = 102
output_size = 1
max_gradient_value = 100_000
max_grad_norm = 40_000

learning_rate = 0.001
num_epochs = 3
batch_size = 32

# Создание простой модели с одним линейным слоем
model = nn.Sequential(
    nn.Linear(input_size, hiden_size),
    nn.ReLU(),
    nn.Linear(hiden_size, output_size)
)
model.to('cuda')
# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
num_samples = X_train.shape[0]
# Обучение модели
counter = 0
for epoch in range(num_epochs):
    for batch_start in range(0, num_samples, batch_size):
        batch_X = X_train[batch_start:batch_start+batch_size]
        batch_y = y_train[batch_start:batch_start+batch_size]

        train_pred = model(batch_X)
        train_loss = criterion(train_pred, batch_y)

        optimizer.zero_grad()
        train_loss.backward()
        
        #model.apply(trim_to_hight)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        
        '''for param in model.parameters():
            if param.grad is not None:
                param.grad.data = torch.clamp(param.grad.data, min=-max_gradient_value, max=max_gradient_value)'''
        
        optimizer.step()

        
        if counter < 11:
            print('-'*40)
            print('Iter: ', counter)
            for name, param in model.named_parameters():
                if param.requires_grad:
                    print(name, param.grad[0])
            
           

        counter += 1


with torch.no_grad():
    test_output = model(X_test)
    print("R2 score: ", r2_score(test_output.cpu().detach().numpy(), y_test.cpu().detach().numpy()))

----------------------------------------
Iter:  0
0.weight tensor([ -26.8180,  399.7422, -204.8105,  -34.1039,   67.8279,   85.6962,
           0.0000,    0.0000,    0.0000,   65.8052,   29.9706,    0.0000,
           0.0000,   87.6014,    0.0000,    0.0000,   83.4022,    0.0000,
          26.1604,    0.0000,   42.4356,    0.0000,    0.0000,  181.8023,
           0.0000,    0.0000,    0.0000,    0.0000,    0.0000,   56.4382,
         200.2783,   67.0695,   27.3202,   19.8205,    0.0000,    0.0000,
          24.5594,    0.0000,   31.7424,    0.0000,   23.1697,   18.1273,
           0.0000,    0.0000,    0.0000,   26.1098,    0.0000,   24.7529,
           0.0000, 1133.6517,   56.4382], device='cuda:0')
0.bias tensor(1190.0901, device='cuda:0')
2.weight tensor([-2761.2705,  -410.7345, -1524.2365, -2012.7534,  -366.8712, -5035.1782,
        -3505.3779, -2923.7246,  -214.6360, -2144.6455, -2346.6177, -4371.4302,
        -1210.0581,  -571.7864, -2338.4189, -3969.1528, -3076.8430, -4571.2661,

Происходит gradient explosere 
Помогает уменьшение lr

теперь понятно почему большое кол-во эпох заставляло работать старые итерации тк на первых двух батчах градиенты еще нормальные и видимо по этому с каждой эпохой первые пару батчей обучали сеть, а остальные взрывали

и в основном проблема в скрытом слое

## Попробую поменять ф активации

In [126]:
def trim_to_hight(layer):
    if isinstance(layer, nn.Linear):
        max_val = 100_000
        min_val = -max_val
        
        layer.weight.data = torch.where(layer.weight.data > max_val, torch.tensor(max_val), layer.weight.data)
        layer.weight.data = torch.where(layer.weight.data < min_val, torch.tensor(min_val), layer.weight.data)
        layer.bias.data = torch.where(layer.bias.data > max_val, torch.tensor(max_val), layer.bias.data)
        layer.bias.data = torch.where(layer.bias.data < min_val, torch.tensor(min_val), layer.bias.data)
 


input_size = 51
hiden_size = 102
output_size = 1
max_gradient_value = 100_000
max_grad_norm = 100_000

learning_rate = 1
num_epochs = 3
batch_size = 16

# Создание простой модели с одним линейным слоем
model = nn.Sequential(
    nn.Linear(input_size, hiden_size),
    nn.Tanh(),
    nn.Linear(hiden_size, output_size)
)
model.to('cuda')
# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
num_samples = X_train.shape[0]
# Обучение модели
counter = 0
for epoch in range(num_epochs):
    for batch_start in range(0, num_samples, batch_size):
        batch_X = X_train[batch_start:batch_start+batch_size]
        batch_y = y_train[batch_start:batch_start+batch_size]

        train_pred = model(batch_X)
        train_loss = criterion(train_pred, batch_y)

        optimizer.zero_grad()
        train_loss.backward()
        
        #model.apply(trim_to_hight)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        
        '''for param in model.parameters():
            if param.grad is not None:
                param.grad.data = torch.clamp(param.grad.data, min=-max_gradient_value, max=max_gradient_value)'''
        
        optimizer.step()

        
        if counter < 11:
            print('-'*40)
            print('Iter: ', counter)
            for name, param in model.named_parameters():
                if param.requires_grad:
                    print(name, param.grad[0])
            
           

        counter += 1


with torch.no_grad():
    test_output = model(X_test)
    print("R2 score: ", r2_score(test_output.cpu().detach().numpy(), y_test.cpu().detach().numpy()))

----------------------------------------
Iter:  0
0.weight tensor([-1026.3396, -2054.3787,   723.4770,  1713.2810,     0.0000,  -476.1562,
            0.0000,  -500.8358,     0.0000,     0.0000,     0.0000,     0.0000,
            0.0000,  -478.4668,     0.0000,     0.0000,     0.0000,     0.0000,
         -148.5792,     0.0000,  -233.6644,     0.0000,     0.0000, -1009.0563,
            0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
        -1135.6807,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
         -137.9588,     0.0000,     0.0000,     0.0000,  -132.3902,  -103.5905,
         -426.3408,     0.0000,     0.0000,  -146.0154,     0.0000,   -70.6513,
            0.0000, -4999.3872,     0.0000], device='cuda:0')
0.bias tensor(-4999.3862, device='cuda:0')
2.weight tensor([   938.4731,  13495.1768,   5975.5552, -12403.9746,  10084.6768,
           310.0079,  -2716.9856,   9191.8115,   7386.6689,  -2960.9941,
          7989.9429,   6156.2632,  -3628.

Тут наверное проблема в том, что у 1го слоя абсольютоные значения меньше чем у 2го

# Теперь попробую отмаштабировать y.
Хотя мне всегда казалось, что так делать не надо 

In [143]:
'''mean = y_train[num_cols].unsqueeze(1).mean()
std = y_train[num_cols].unsqueeze(1).std()

# Standardize numerical features
y_train[num_cols] = (y_train[num_cols] - mean) / std
y_test[num_cols] = (y_test[num_cols] - mean) / std
'''
#print(y_test.mean(dim=0), y_test.std(dim=0))
#print(y_train.mean(dim=0), y_train.std(dim=0))

y_train = (y_train - y_test.mean(dim=0)) / y_test.std(dim=0)
y_test = (y_test - y_train.mean(dim=0)) / y_train.std(dim=0)


Проверим теперь трехслойную с ReLU и потом Tanh - неполучилось (( 
    
добавления Tanh на выходе тоже не помогло 

In [159]:
def trim_to_hight(layer):
    if isinstance(layer, nn.Linear):
        max_val = 100_000
        min_val = -max_val
        
        layer.weight.data = torch.where(layer.weight.data > max_val, torch.tensor(max_val), layer.weight.data)
        layer.weight.data = torch.where(layer.weight.data < min_val, torch.tensor(min_val), layer.weight.data)
        layer.bias.data = torch.where(layer.bias.data > max_val, torch.tensor(max_val), layer.bias.data)
        layer.bias.data = torch.where(layer.bias.data < min_val, torch.tensor(min_val), layer.bias.data)
 


input_size = 51
hiden_size = 102
output_size = 1
max_gradient_value = 100_000
max_grad_norm = 40_000

learning_rate = 0.01
num_epochs = 3
batch_size = 32

# Создание простой модели с одним линейным слоем
model = nn.Sequential(
    nn.Linear(input_size, hiden_size),
    nn.ReLU(),
    nn.Linear(hiden_size, output_size),
    nn.Tanh()
)
model.to('cuda')
# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
num_samples = X_train.shape[0]
# Обучение модели
counter = 0
for epoch in range(num_epochs):
    for batch_start in range(0, num_samples, batch_size):
        batch_X = X_train[batch_start:batch_start+batch_size]
        batch_y = y_train[batch_start:batch_start+batch_size]

        train_pred = model(batch_X)
        train_loss = criterion(train_pred, batch_y)

        optimizer.zero_grad()
        train_loss.backward()
        
        #model.apply(trim_to_hight)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        
        '''for param in model.parameters():
            if param.grad is not None:
                param.grad.data = torch.clamp(param.grad.data, min=-max_gradient_value, max=max_gradient_value)'''
        
        optimizer.step()

        print(train_loss)
        '''if counter < 11:
            print('-'*40)
            print('Iter: ', counter)
            for name, param in model.named_parameters():
                if param.requires_grad:
                    print(name, param.grad[0])'''
            
           

        counter += 1


with torch.no_grad():
    test_output = model(X_test)
    print("R2 score: ", r2_score(test_output.cpu().detach().numpy(), y_test.cpu().detach().numpy()))

tensor(1.5190, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.9515, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(1.1274, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(1.5977, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.7867, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(1.2732, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.6696, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(1.2742, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(1.0564, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(1.1175, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(1.3322, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(1.0062, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(1.0932, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.9051, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.7662, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(1.2496, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.7983, device='cuda:0', grad_fn=